In [1]:
#get the FOC given a function
from sympy import *
init_printing(use_latex='mathjax')

x, f= symbols('x f')
f = diff(-(x-4)**2, x)
print('first differential of', -(x-4)**2, f)

first differential of -(x - 4)**2 -2*x + 8


In [2]:
#get the maximimu or minimum point 
print('set the first differential equation to 0, we get:', solve(f, x), '\nas maximium point')

set the first differential equation to 0, we get: [4] 
as maximium point


In [3]:
#another method 
print('Numerical solution searching, we get:')
from scipy import optimize as opt
def f(x):
    return (x-4)**2
#opt.minimize_scalar(f, method='Brent')
opt.minimize_scalar(f, method='bounded', bounds=[0, 8])

Numerical solution searching, we get:


     fun: 0.0
 message: 'Solution found.'
    nfev: 6
  status: 0
 success: True
       x: 4.0

In [4]:
#for C2 function with multiple xi variables
# Check Hessian to see if there is critical point
import numpy as np
from sympy import symbols, hessian, Function, N

x1, x2 = symbols('x1 x2')
f = symbols('f', cls=Function)

f = -((x1-1)**2 + (x2)**2)
H = hessian(f, [x1, x2]).subs([(x1,1), (x2,1)])
print('Hessian:\n', np.array(H))

Hessian:
 [[-2 0]
 [0 -2]]


In [5]:
#find the numerical solution to the problem with constraints by using computer search algorithm.
#constraint: x1+x2 <=-2
print('numerical solution for the equivalent constrained minimization problem:\n')
cons = ({'type': 'ineq',
         'fun' : lambda x: np.array([-x[1] - x[0] - 2])})

def f(x):
    return ((x[0]-1)**2 + (x[1])**2)
cx = opt.minimize(f, [1, -2], constraints=cons)#program flips the function and constraint and do a minimization
cx#the result is the same with the solution from lagrangian solution

numerical solution for the equivalent constrained minimization problem:



     fun: 4.500000421997264
     jac: array([-3.00091863, -2.99908125])
 message: 'Optimization terminated successfully.'
    nfev: 18
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([-0.50045935, -1.49954065])

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from numpy.linalg import inv
from numpy import dot
#write a function to solve simplest mean-variance problem, in which rf is considered to be 0
def meanvar(miu,rbar,cov):
    """
    this function is used to get the minimal variance
    miu is the expected return for a given set of asset
    rbar is the expected return vector for a set of assets 
    cov is the covariance matrix for the assets
    return[0] is the vector of weights and return[1] is the minimized standard deviation
    """
    import math
    ones = np.ones(rbar.shape)
    #solve lagrangian lamda 1,2 from fomulas
    A=rbar.T.dot(inv(cov)).dot(rbar)
    B=rbar.T.dot(inv(cov)).dot(ones)
    C=ones.T.dot(inv(cov)).dot(ones)    
    lambda1=(C*miu-B)/(A*C-B**2)
    lambda2=(A-B*miu)/(A*C-B**2)
    #get the vector of weights
    omiga=(inv(cov)).dot(rbar)*lambda1+(inv(cov)).dot(ones)*lambda2
    #get the minimized standard deviation
    GMVstd=math.sqrt((A-2*B*miu+C*miu**2)/(A*C-B**2))
    #return the result
    return omiga,GMVstd
    

In [ ]:
#Plot mean-variance efficeint curve. 
mus = np.linspace(0.001, 0.12, 100) 
sigmas =[] 
for m in mus:    
    sm = meanvar(m,rbar,cov)[1]    
    sigmas.append(sm) 
zipped =zip(mus, sigmas) 
fig, ax = plt.subplots(figsize=(16,10))
plt.plot(sigmas, mus, label='Mean Variance Frontier') 
plt.xlabel(r'$\sigma (R)$') 
plt.ylabel(r'$E(R)$') 
plt.title('Mean Variance Frontier',size=15) 
plt.xlim([0, 0.25]) 
plt.legend()
plt.show()  

In [ ]:
#write a function to solve mean-variance problem given risk free rate
def meanvar2(rbar,cov,rf):
    """
    this function is used to get the weights which allow us to have the maximal sharpe ratio
    rbar is the expected return vector for a set of assets 
    cov is the covariance matrix for the assets
    rf is the given risk free rate
    return[0] is the vector of weights 
    """
    import math
    ones = np.ones(rbar.shape)
    numerator=inv(cov).dot(rbar-rf*ones)
    denominator=(rbar-rf*ones).T.dot(inv(cov)).dot(ones)
    omiga=numerator/denominator
    return omiga